In [63]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from synthetics import generate_single_task_unipolar, gaussian_bags_of_words, vocab1k

N = 1000
M = 30
K = 2
NUM_SPLITS = 3

Ds = [] # data
Xs = [] # features
Ls = [] # noisy labels
Ys = [] # true labels
for _ in range(NUM_SPLITS):
    L, Y, _ = generate_single_task_unipolar(
        N, M, k=K, alpha_range=[0.6, 0.9], beta_range=[0.1, 0.2], 
        class_balance=[0.3, 0.7], polarity_balance=None, seed=1)
    
    X, D = gaussian_bags_of_words(Y, vocab1k)
    
    Ls.append(L)
    Ys.append(Y)
    Ds.append(D)
    Xs.append(X)

In [42]:
search_space = {
    'n_epochs': 300,
    'l2': [0.0, 0.01, 0.1],
    'lr': {'range': [0.0001, 1], 'scale': 'log'},
}

In [43]:
# First run basic grid search / random search
from metal.label_model import LabelModel
from metal.tuner import ModelTuner

tuner = ModelTuner(LabelModel)
init_args = []
train_args = [Ls[0]]
model, best_config = tuner.search(init_args, train_args, Ls[1], Ys[1], 
                                  search_space, max_search=5, metric='f1')

[1] Testing {'l2': 0.1, 'lr': 0.00823215577156734}
Computing O...
Estimating \mu...
[Epoch 0] Loss: 52.976555
[Epoch 10] Loss: 1.651653
[Epoch 20] Loss: 2.710884
[Epoch 30] Loss: 0.082714
[Epoch 40] Loss: 0.194947
[Epoch 50] Loss: 0.045159
[Epoch 60] Loss: 0.013377
[Epoch 70] Loss: 0.018542
[Epoch 80] Loss: 0.010708
[Epoch 90] Loss: 0.007583
[Epoch 100] Loss: 0.007214
[Epoch 110] Loss: 0.006621
[Epoch 120] Loss: 0.006258
[Epoch 130] Loss: 0.006073
[Epoch 140] Loss: 0.005927
[Epoch 150] Loss: 0.005811
[Epoch 160] Loss: 0.005716
[Epoch 170] Loss: 0.005635
[Epoch 180] Loss: 0.005561
[Epoch 190] Loss: 0.005493
[Epoch 200] Loss: 0.005429
[Epoch 210] Loss: 0.005368
[Epoch 220] Loss: 0.005309
[Epoch 230] Loss: 0.005250
[Epoch 240] Loss: 0.005192
[Epoch 250] Loss: 0.005135
[Epoch 260] Loss: 0.005077
[Epoch 270] Loss: 0.005020
[Epoch 280] Loss: 0.004963
[Epoch 290] Loss: 0.004906
[Epoch 299] Loss: 0.004855
F1: 0.622
[2] Testing {'l2': 0.01, 'lr': 0.10760132314343232}
Computing O...
Estimating \

In [65]:
# Hyperband
from metal.label_model import LabelModel
from metal.tuner import HyperbandTuner

tuner = HyperbandTuner(LabelModel, seed=456, hyperband_epochs_budget=200)
init_args = []
train_args = [Ls[0]]
model, best_config = tuner.search(init_args, train_args, Ls[1], Ys[1], 
                                  search_space, metric='f1')

|           Hyperband Schedule          |
Table consists of tuples of (num configs, num_resources_per_config) which specify how many configs to run and for how many epochs. 
Each bracket starts with a list of random configurations which is successively halved according the schedule.
See the Hyperband paper (https://arxiv.org/pdf/1603.06560.pdf) for more details.
-----------------------------------------
Bracket 0: (9, 2) (3, 8) (1, 26)
Bracket 1: (3, 8) (1, 26)
Bracket 2: (3, 26)
-----------------------------------------
9
[0 Testing {'l2': 0.0, 'lr': 0.7351047587103773}
Computing O...
Estimating \mu...
[Epoch 0] Loss: 45.158043
[Epoch 1] Loss: 55489.750000
F1: 0.404
[1 Testing {'l2': 0.01, 'lr': 0.005506013699464609}
Computing O...
Estimating \mu...
[Epoch 0] Loss: 34.639576
[Epoch 1] Loss: 28.934540
F1: 0.644
[2 Testing {'l2': 0.1, 'lr': 0.15038313076975984}
Computing O...
Estimating \mu...
[Epoch 0] Loss: 57.042675
[Epoch 1] Loss: 12.385881
F1: 0.136
[3 Testing {'l2': 0.0, 'lr': 0.0